# [Classify ImageNet classes with ResNet50](https://keras.io/applications/#usage-examples-for-image-classification-models)

In [1]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

# To prevent an issue similar to https://stackoverflow.com/questions/42969779/keras-error-you-must-feed-a-value-for-placeholder-tensor-bidirectional-1-keras
from keras import backend as K
K.set_learning_phase(0)

model = ResNet50(weights='imagenet')

img_path = 'cropped_panda.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [('n02510455', 'giant_panda', 0.99036354), ('n02500267', 'indri', 0.0058755828), ('n02509815', 'lesser_panda', 0.0016656154)]

Using TensorFlow backend.


Predicted: [('n02510455', 'giant_panda', 0.99036354), ('n02500267', 'indri', 0.0058755828), ('n02509815', 'lesser_panda', 0.0016656154)]


# Altered [FreezeGraphTest](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/tools/freeze_graph_test.py#L15-L107) with predictions

In [2]:
"""Tests the graph freezing tool."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

from tensorflow.core.example import example_pb2
from tensorflow.core.framework import graph_pb2
from tensorflow.core.protobuf import saver_pb2
from tensorflow.python.client import session
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import graph_io
from tensorflow.python.framework import importer
from tensorflow.python.framework import ops
from tensorflow.python.framework import test_util
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import parsing_ops
from tensorflow.python.ops import variables
from tensorflow.python.platform import test
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import signature_def_utils
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.tools import freeze_graph
from tensorflow.python.training import saver as saver_lib


class FreezeGraphTest(test_util.TensorFlowTestCase):
    
    def _testFreezeGraph(self, saver_write_version):
        
        checkpoint_prefix = os.path.join(self.get_temp_dir(), "saved_checkpoint")
        checkpoint_meta_graph_file = os.path.join(self.get_temp_dir(),
                                                  "saved_checkpoint.meta")
        checkpoint_state_name = "checkpoint_state"
        input_graph_name = "input_graph.pb"
        output_graph_name = "output_graph.pb"

        # # We'll create an input graph that has a single variable containing 1.0,
        # # and that then multiplies it by 2.
        # with ops.Graph().as_default():
        with K.get_session().graph.as_default(): # Get DenseNet50 graph from Keras session
            # variable_node = variables.Variable(1.0, name="variable_node")
            # sess = session.Session()
            sess = K.get_session()
            init = variables.global_variables_initializer()
            sess.run(init)
            # output = sess.run(output_node)
            # self.assertNear(2.0, output, 0.00001)
            saver = saver_lib.Saver(write_version=saver_write_version)
            checkpoint_path = saver.save(
                sess,
                checkpoint_prefix,
                global_step=0,
                latest_filename=checkpoint_state_name)
            graph_io.write_graph(sess.graph, self.get_temp_dir(), input_graph_name)

        # We save out the graph to disk, and then call the const conversion
        # routine.
        input_graph_path = os.path.join(self.get_temp_dir(), input_graph_name)
        input_saver_def_path = ""
        input_binary = False
        output_node_names = model.outputs[0].op.name
        restore_op_name = "save/restore_all"
        filename_tensor_name = "save/Const:0"
        output_graph_path = os.path.join(self.get_temp_dir(), output_graph_name)
        clear_devices = False
        input_meta_graph = checkpoint_meta_graph_file
        
        freeze_graph.freeze_graph(
            input_graph_path, input_saver_def_path, input_binary, checkpoint_path,
            output_node_names, restore_op_name, filename_tensor_name,
            output_graph_path, clear_devices, "", "")#, input_meta_graph) # freeze_graph() got an unexpected keyword argument 'input_meta_graph'

        # Now we make sure the variable is now a constant, and that the graph still
        # produces the expected result.
        with ops.Graph().as_default():
            output_graph_def = graph_pb2.GraphDef()
            with open(output_graph_path, "rb") as f:
                output_graph_def.ParseFromString(f.read())
                _ = importer.import_graph_def(output_graph_def, name="")

            # self.assertEqual(4, len(output_graph_def.node))
            for node in output_graph_def.node:
                self.assertNotEqual("VariableV2", node.op)
                self.assertNotEqual("Variable", node.op)

            with K.get_session() as sess:
                # output_node = sess.graph.get_tensor_by_name("output_node:0")
                input_node = sess.graph.get_tensor_by_name("input_1:0")
                output_node = sess.graph.get_tensor_by_name("fc1000/Softmax:0")
                # output = sess.run(output_node)
                output = sess.run(output_node, feed_dict={input_node: x})
                # self.assertNear(2.0, output, 0.00001)
                print()
                print('Predicted 1:', decode_predictions(preds, top=3)[0])
                print('Predicted 2:', decode_predictions(output, top=3)[0])           

In [3]:
FreezeGraphTest()._testFreezeGraph(saver_pb2.SaverDef.V2)

INFO:tensorflow:Froze 320 variables.
Converted 320 variables to const ops.
1253 ops in the final graph.

Predicted 1: [('n02510455', 'giant_panda', 0.99036354), ('n02500267', 'indri', 0.0058755828), ('n02509815', 'lesser_panda', 0.0016656154)]
Predicted 2: [('n01910747', 'jellyfish', 0.27250421), ('n03995372', 'power_drill', 0.21951485), ('n03787032', 'mortarboard', 0.1852365)]
